# 7-2. 텍스트 감정분석의 유용성

텍스트 감정분석 접근법</br>
- 기계학습 기반 접근법
- 감성사전 기반 접근법
    - 기계학습 대비 다음의 단점들이 있다.
        1. 분석 대상에 따라 단어의 감성 점수가 달라질 수 있다는 가능성에 대응하기 힘듦
        2. 단순 긍부정을 넘어서 긍부정의 원인이 되는 대상 속성 기반의 감성 분석이 어려움
- 데이터분석 업무 측면에서의 텍스트 분류 모델
    - 일반적인 데이터분석 업무에서는 범주화가 잘된 정형데이터를 필요로 함
        - 정형데이터는 규모가 커질수록 비용이 기하급수적으로 증가할 수 있다.
    - 비정형데이터인 텍스트에 감성분석 기법을 사용
        - 텍스트를 정형데이터로 가공하여 유용한 의사결정 보조자료로 활용 가능함


워드 임베딩(word embedding) 기법</br>
- 단어의 특성을 저차원 벡터 값으로 표현하는 기법
- 머신러닝 기반 감성분석 시 비용을 절감할 수 있음
    - 머신러닝 추론 일치도(정확도)를 크게 향상할 수 있음

# 7-4. 텍스트 데이터의 특징
## (1) 텍스트를 숫자로 표현하는 방법

단어와 그 **단어의 의미를 나타내는 벡터**를 짝지어 보자.</br>
> 마치 사전에서 특정 단어와 그 단어에 대한 설명이 짝지어진 것 처럼

텍스트 데이터를 처리하는 예제
```text
## 텍스트 데이터 예시

    i feel hungry
    i eat lunch
    now i feel happy
```

In [1]:
# 문장 입력 및 공백을 기준으로 split
# 처리해야 할 문장을 파이썬 리스트에 옮겨 담았습니다.
sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']

# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개 봅니다.
word_list = 'i feel hungry'.split()
print(word_list)

['i', 'feel', 'hungry']


In [2]:
# 텍스트 데이터를 기반으로 사전을 만들기 위한 예시
## python dictionary 활용

index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

{0: '<PAD>', 1: '<BOS>', 2: '<UNK>', 3: 'i', 4: 'feel', 5: 'hungry', 6: 'eat', 7: 'lunch', 8: 'now', 9: 'happy'}


In [3]:
# 텍스트 데이터를 숫자로 변환하기
## 예제 기준에서 key와 value를 바꿔버리면 된다.
word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

{'<PAD>': 0, '<BOS>': 1, '<UNK>': 2, 'i': 3, 'feel': 4, 'hungry': 5, 'eat': 6, 'lunch': 7, 'now': 8, 'happy': 9}


In [4]:
# 바뀐 단어의 값(숫자)을 확인
print(word_to_index['feel'])  # 단어 'feel'은 숫자 인덱스 4로 바뀝니다.

4


In [5]:
# 가지고 있는 텍스트 데이터들을 숫자로 바꿔 표현하기
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수를 만들어 봅시다.
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

print(get_encoded_sentence('i eat lunch', word_to_index))

[1, 3, 6, 7]


In [6]:
# 텍스트 데이터 사전화 예시
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 가 아래와 같이 변환됩니다. 
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]]


In [7]:
# 숫자 값으로 표현된 단어들을 다시 원래 단어(sentence)로 복구하기
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

i feel hungry


In [8]:
# 문장 encode, decode 예시
# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

# encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 가 아래와 같이 변환됩니다.
print(get_decoded_sentences(encoded_sentences, index_to_word))

['i feel hungry', 'i eat lunch', 'now i feel happy']


# 7-5. 텍스트 데이터의 특징
## (2) Embedding 레이어의 등장
임베딩(Embedding)?
```text
자연어 처리(Natural Language Processing)분야에서 말하는 임베딩(Embedding)
- 사람이 쓰는 자연어를 기께가 이해할 수 있는 숫자형태(vector)로 바꾼 결과
- 혹인 그 일련의 과정 전체
```
What to do with Embedding?
```text
단어나 문장 사이의 코사인 유사도가 가장 높은 단어 구하기 (계산)
단어들 사이의 의미/문법적 정보 도출
단어 사이 문법적 관계 도출 (벡터 간 연산)
```
> 전이 학습(transfer learning) 임베딩
>> 다른 딥러닝 모델의 입력값으로 자주 쓰임
>> 품질 좋은 임베딩을 사용할수록 모델의 성능이 좋아짐
[임베딩 레이어를 통해 word to vector](https://wikidocs.net/64779)

In [9]:
# word to vector with tensorflow 예제
# 아래 코드는 그대로 실행하시면 에러가 발생할 것입니다. 

import numpy as np
import tensorflow as tf
import os

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 위 그림과 같이 4차원의 워드 벡터를 가정합니다. 

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# 숫자로 변환된 텍스트 데이터 [[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 에 Embedding 레이어를 적용합니다. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype='object')
output = embedding(raw_inputs)
print(output)

2023-07-13 15:01:54.224446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

### Embedding Layer 주의점
Embedding 레이어의 인풋 문장 벡터의 **길이는 일정**해야한다.</br>
> raw_inputs에 있는 3개 베거의 길이는 각각 다음과 같다.
>> 4, 4, 5

In [10]:
# TF의 함수 사용하여 input 벡터 길이 맞추기
## tf.keras.preprocessing.sequence.pad_sequences() 사용
## 문장 벡터 뒤에 패팅(<PAD>)을 추가하여 길이를 일정하게 맞춰줌
raw_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
print(raw_inputs)

[[1 3 4 5 0]
 [1 3 6 7 0]
 [1 8 3 4 9]]


> <PAD>에는 0과 매핑되어 있다.

In [11]:
# input 벡터들 (raw_inputs)의 길이를 맞춰주면서 word to vector를 수행하는 코드
vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 그림과 같이 4차원의 워드 벡터를 가정합니다.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# tf.keras.preprocessing.sequence.pad_sequences를 통해 word vector를 모두 일정 길이로 맞춰주어야 
# embedding 레이어의 input이 될 수 있음에 주의해 주세요. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype=object)
raw_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
output = embedding(raw_inputs)
print(output)

tf.Tensor(
[[[ 0.03563546 -0.02319768 -0.01442961  0.02279365]
  [ 0.03037668 -0.04669473 -0.00695539 -0.04383923]
  [ 0.03651122 -0.04469563 -0.02924101  0.01164202]
  [-0.0459154  -0.03801433 -0.02539301 -0.0061772 ]
  [-0.00923066  0.00127262  0.03522557 -0.02817357]]

 [[ 0.03563546 -0.02319768 -0.01442961  0.02279365]
  [ 0.03037668 -0.04669473 -0.00695539 -0.04383923]
  [ 0.00056914  0.00822072 -0.00984446  0.02501208]
  [-0.04352775 -0.00191113  0.00325835  0.04944073]
  [-0.00923066  0.00127262  0.03522557 -0.02817357]]

 [[ 0.03563546 -0.02319768 -0.01442961  0.02279365]
  [-0.00036765  0.00811856 -0.0046363  -0.04263892]
  [ 0.03037668 -0.04669473 -0.00695539 -0.04383923]
  [ 0.03651122 -0.04469563 -0.02924101  0.01164202]
  [ 0.0451659  -0.03582686 -0.01209273 -0.04598297]]], shape=(3, 5, 4), dtype=float32)


> output의 shape(3, 5, 4)에서 각 값의 의미
>> 3 : 입력문장 개수
>> 5 : 입력문장의 최대 길이
>> 4 : 워드 벡터의 차원 수

# 7-6. 시퀀스 데이터를 다루는 RNN
> 텍스트 데이터를 다루는데 주로 사용되는 딥러닝 모델
>> Recurrent Neural Network (RNN)
>> 시퀀스(Sequence) 형태의 데이터를 처리하기에 최적인 모델로 알려져 있음

[stateful vs stateless 예시; Web의 관점에서](https://www.slideshare.net/xguru/ss-16106464)</br>
[모두의 딥러닝 강좌 - 12강.RNN, 김성훈 교수](https://youtu.be/-SHPG_KMUkQ)

In [12]:
# RNN 모델을 사용하여 텍스트 데이터를 처리하는 예제 코드
## 텍스트 데이터는 이전의 텍스트를 의미
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 4)           40        
                                                                 
 lstm (LSTM)                 (None, 8)                 416       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 537 (2.10 KB)
Trainable params: 537 (2.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> 시퀀스 데이터와 RNN 참고자료
[Youtube Link](https://youtu.be/mG6N0ut9dog?t=1447)

# 7-7. 꼭 RNN이어야 할까?
텍스트 처리에서는 RNN이 아니라 `1-D Convolution Neural Network (1-D CNN)`를 사용할수도 있다.</br>
```text
이전에 이미지 분류기 구현에서 `2-D CNN`을 사용해보았다.

 - 이미지는 시퀀스 데이터가 아니다.

따라서 이미지 분류기 모델에서 사용된 입력은 이미지 전체다.
```
> 1-D CNN (with text data)
>> 문장 전체를 한꺼번에 한 방향으로 스캐닝
>> 한 번에 처리하는 양은 7 (len 7)
>> 7 단어 이내에서 발견되는 특징을 추출하여 추출결과값을 문장 분류에 사용함

`1-D CNN`방식도 텍스트 처리에서 RNN 못지않은 효율을 보여줌</br>
> CNN 계열의 이점
>> RNN 계열보다 병렬처리가 효율적이다
>> 학습 속도가 RNN에 빠르게 진행된다.

In [13]:
# 1-D CNN을 활용하여 텍스트 데이터 처리 및 모델 학습
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 4)           40        
                                                                 
 conv1d (Conv1D)             (None, None, 16)          464       
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 16)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 16)          1808      
                                                                 
 global_max_pooling1d (Glob  (None, 16)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense_2 (Dense)             (None, 8)                

In [14]:
# 1-D CNN 기법에서 GlobalMaxPooling() 레이어만 사용하는 예제
## desc.) 전체 문장 중에서 단 하나의 가장 중요한 단어만 feature로 추출
## 추출된 feature로 문장의 궁/부정을 평가
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 4)           40        
                                                                 
 global_max_pooling1d_1 (Gl  (None, 4)                 0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 8)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 89 (356.00 Byte)
Trainable params: 89 (356.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> 이 외에도 고려할만한 방법
>> 1-D CNN과 RNN 섞어 쓰기
>> FFN (FeedForward Network) 레이어만 사용해보기
>> Transformer 레이어 사용해보기

[참고링크](https://wikidocs.net/80437)